# **Training on LSUN (Large Scale Understanding) Bedrooms Data Set**

In [1]:
import os
from miniai.imports import *
from miniai.diffusion import *
from diffusers import UNet2DModel, UNet2DConditionModel, AutoencoderKL
from fastprogress import progress_bar
from glob import glob
from copy import deepcopy
import timm

In [2]:
torch.set_printoptions(precision=4, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray_r'
mpl.rcParams['figure.dpi'] = 70

set_seed(42)
if fc.defaults.cpus>8: fc.defaults.cpus=8

## **Download and Process Data**

In [ ]:
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path = path_data/'bedroom'

In [ ]:
url = 'https://s3.amazonaws.com/fast-ai-imageclas/bedroom.tgz'
if not path.exists():
    path_zip = fc.urlsave(url, path_data)
    shutil.unpack_archive('data/bedroom.tgz', 'data')

In [ ]:
bs = 64

In [ ]:
def to_img(f): return read_image(f, mode=ImageReadMode.RGB)/255

In [ ]:
class ImagesDS:
    def __init__(self, spec):
        self.path = Path(path)
        self.files = glob(str(spec), recursive=True)

    def __len__(self): return len(self.files)

    def __getitem__(self, i): return to_img(self.files[i])[:, :256, :256]

In [ ]:
ds = ImageDS(path/f'**/*.jpg')

In [ ]:
dl = DataLoader(ds, batch_size=bs, num_workers=fc.defaults.cpus)
xb = next(iter(dl))
show_images(xb[:16], imsize=2)

In [ ]:
xb[:16].shape

In [3]:
16*3*256*256

3145728

## **Using a Pre-Trained VAE**

In [ ]:
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-ema").cuda().requires_grad_(False)